In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

from torch.utils.data import random_split, DataLoader
from torchvision import transforms, datasets

import os

In [9]:
%matplotlib inline

In [10]:
valid_split = 0.2
test_split = 0.2

initial_transforms = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='garbage_dataset/garbage_images', transform=initial_transforms)
valid_size = int(0.2 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size = len(dataset) - valid_size - test_size

train_set, valid_set, test_set = random_split(dataset, [train_size, valid_size, test_size])

In [11]:
# Calculate mean and std of training set
data_r = np.dstack([np.array(train_set[i][0])[0, :, :] for i in range(len(train_set))])  
data_g = np.dstack([np.array(train_set[i][0])[1, :, :] for i in range(len(train_set))])    
data_b = np.dstack([np.array(train_set[i][0])[2, :, :] for i in range(len(train_set))])    

mean_train = (np.mean(data_r), np.mean(data_g), np.mean(data_b))
std_train = (np.std(data_r), np.std(data_g), np.std(data_b))

In [12]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_train, std=std_train)
])

dataset = datasets.ImageFolder(root='garbage_dataset/garbage_images', transform=data_transform)
valid_size = int(0.2 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size = len(dataset) - valid_size - test_size

train_set, valid_set, test_set = random_split(dataset, [train_size, valid_size, test_size])


train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=16, shuffle=True, num_workers=4)


In [13]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print_every = 100

print('using device:', device)

using device: cpu


In [14]:
def train(model, optimizer, epochs=1):
    """
    Train the model on training data.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(valid_loader, model)
                print()

In [15]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [16]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [17]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = nn.Sequential(
    nn.Conv2d(3, channel_1, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(384*512*channel_2, 10)
)

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train(model, optimizer, epochs=10)

Iteration 0, loss = 2.2949
Got 89 / 505 correct (17.62)

Iteration 0, loss = 1.9438
Got 123 / 505 correct (24.36)

Iteration 0, loss = 1.8814
Got 123 / 505 correct (24.36)

Iteration 0, loss = 2.0368
Got 123 / 505 correct (24.36)



Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 